In [ ]:
#-*- encoding: utf8 -*-
%matplotlib inline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import json
import zipfile
from tqdm import tqdm
from matplotlib import pyplot as plt
import ast
# Any results you write to the current directory are saved as output.

In [ ]:
!mkdir ../working/main_data

In [ ]:
./

# 데이터 구성 분석
* drawing is consisit of [x,y] point and start, end 2d 형식으로 구성
* recognized 는 해당 drawing에 대한 진위 여부

In [ ]:
SEED = 42
INPUT_DIR = "../input/"
train_simple_eiffel = pd.read_csv(os.path.join(INPUT_DIR, 'train_simplified/The Eiffel Tower.csv'))
train_simple_eiffel.head()

In [ ]:
train_simple_eiffel['recognized'].value_counts()

## Matplot 시각화를 통한 데이터 형태 분석
* Line이 작성된 순서에 따른 Sequential한 성질을 보임(Sequential 특징이 반영된 모델 추가 구성)
* recognized False 케이스는 대체로 관련성 없는 형태의 그림을 보임
* Evaluate Metric이 Precision에 집중된 Task, 따라서 True에 해당하는 데이터를 중심으로 학습 진행


In [ ]:
train_simple_eiffel['drawing'] = train_simple_eiffel['drawing'].apply(ast.literal_eval)

In [ ]:
n = 10
fig, axs = plt.subplots(nrows=n, ncols=n, sharex=True, sharey=True, figsize=(16, 10))
for i, drawing in enumerate(train_simple_eiffel[train_simple_eiffel['recognized']==True][:100].drawing):
    ax = axs[i // n, i % n]
    for x, y in drawing:
        ax.plot(x, -np.array(y), lw=3)
    ax.axis('off')
fig.savefig('eiffe_true.png', dpi=200)
plt.show();

In [ ]:
n = 10
fig, axs = plt.subplots(nrows=n, ncols=n, sharex=True, sharey=True, figsize=(16, 10))
for i, drawing in enumerate(train_simple_eiffel[train_simple_eiffel['recognized']==False][:100].drawing):
    ax = axs[i // n, i % n]
    for x, y in drawing:
        ax.plot(x, -np.array(y), lw=3)
    ax.axis('off')
fig.savefig('eiffe_false.png', dpi=200)
plt.show();

# Train,Test data 생성
* 340개의 카테고리를 KEYID 기준으로 Split을 통하여 100개의 categories mixed 데이터셋 생성
* recognized True인 데이터 셋만 가져옴

In [ ]:
CHUNK_DIR = '../working/main_data/'

In [ ]:
np.random.seed(seed = SEED)

In [ ]:
len(os.listdir(os.path.join(INPUT_DIR, 'train_simplified/')))

In [ ]:
print(len(os.listdir(os.path.join(INPUT_DIR, 'train_simplified/'))))

In [ ]:
def file2cat(filename):
    return filename.split('.')[0]

In [ ]:
class Simplified():
    def __init__(self, input_path = '../input'):
        self.input_path = input_path
    def list_all_categories(self):
        files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        return sorted([file2cat(f) for f in files], key = str.lower)
    def read_training_csv(self, category, nrows = None, usecols = None, drawing_transform = False):
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),\
                        parse_dates = ['timestamp'],usecols=usecols)
        df = df[df['recognized']==True].reset_index(drop=True)
        df = df.iloc[:nrows]
        
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df

In [ ]:
s = Simplified(INPUT_DIR)
DATA_CHUNK = 100
categories = s.list_all_categories()
print(len(categories))

In [ ]:
for y, cat in tqdm(enumerate(categories)):
    df = s.read_training_csv(cat, nrows=30000)
    df['y'] = y
    df['cv'] = (df.key_id//10 ** 7) % DATA_CHUNK
    for k in range(DATA_CHUNK):
        filename = 'train_k{}.csv'.format(k)
        filename = os.path.join(CHUNK_DIR, filename)
        chunk = df[df.cv==k]
        chunk = chunk.drop(['key_id'],axis = 1)
        if y==0:
            chunk.to_csv(filename, index =False)
        else:
            chunk.to_csv(filename, mode = 'a', header = False, index = False)

In [ ]:
import gc
from multiprocessing import Pool
from functools import partial
from itertools import repeat
from itertools import product

## Multiprocessing을 통한 병렬처리

In [ ]:
!grep -c processor /proc/cpuinfo

In [ ]:
NJOBS = !grep -c processor /proc/cpuinfo

In [ ]:
def chunk2gzip(chunk_number):
#     for k in range(chunk_number,JOBS):
#         try:
    filename = 'train_k{}.csv'.format(chunk_number)
    filename = os.path.join(CHUNK_DIR, filename)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        print(df.shape)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by ='rnd').drop('rnd', axis =1)
        df.to_csv(filename + '.gz', compression='gzip', index = False)
        os.remove(filename)
#         except IndexError:
#             print("file no. {} not in".format(k))
#             pass
    

In [ ]:
with Pool(processes=int(NJOBS[0])) as p:
    max_ = DATA_CHUNK
    with tqdm(total=max_) as pbar:
        for i, _ in tqdm(enumerate(p.imap(chunk2gzip, list(range(0,100))))):
            pbar.update()
